# Task 4

# Task 4.1

Реализовать генератор матрциц, который должен поддерживать функции:
* Генерация абсолютно случайной матрицы $n\times m$
* Генерация случайной диагональной матрицы $n\times n$
* Генерация случайной верхнетреугольной матрицы
* Генерация случайной нижнетреугольной матрицы
* Генерация симметричной матрицы
* Генерация вырожденной матрицы
* Генерация матрицы ступенчатого вида $n\times n$ ранга $m$
* Генерация возмущения матрицы $n\times m$, каждый элемент которой не превосходит по модулю заданный $\varepsilon$. Оценить величину нормы матрицы возмущений в зависимости от параметра $\varepsilon$ (оценить верхную границу).

Оценить численно вероятность того, что созданная матрица будет вырожденной для какого-либо случая выше. 

In [283]:
import numpy as np

def matrix_generate(rows, columns, type_ = "full", eps = 0):
    """
    matrix_generate(rows, columns, type_ = "full")
    
    Создаёт случайную матрицу выбранного типа. 
    
    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".
    
    Parameters
    ----------
    
    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "upper_triangular", "symmetric" и т.д.
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц.
    
    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.
        
    Notes
    -----
    Поддерживаемые типы матриц:
        "full","upper_triangular",
        "symmetric",
        ...
    
    
    """
    
    A = None

    def jordan(n, a):
        A = np.eye(n, n)*a
        for i in range(n-1):
            A[i, i+1] = 1
        return A
    
    if type_ == "full":
        
        A = np.random.random(size=(rows, columns))
        
    elif type_ == "upper_triangular":
        
        A = np.random.random(size=(rows, columns))

        for i in range(rows):
            for j in range(columns):
                if (i > j):
                    A[i, j] = 0
    
        # Для нижнетреугольной подумайте, как сделать без циклов for :) (звёздочка)
    
    elif type_ == "lower_triangular":
        
        A = np.random.random(size=(rows, columns))

        for i in range(rows):
            for j in range(columns):
                if (i < j):
                    A[i, j] = 0

    elif type_ == "diag":

        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"

        A = np.random.random(size=(rows, columns))
        A = np.diag(np.diag(A))

    elif type_ == "symmetric":
        
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        else:
            
            A = np.random.random(size=(rows, columns))
        
            for i in range(rows):
                for j in range(columns):
                    if (i > j):
                        A[i, j] = A[j, i]
                        
            # И эту секцую тоже перепишите без for (звёздочка). Учтите, что портить uniform распределение нельзя.

    # elif type_ == 'stup':
    #     A = np.random.random(size=(rows, rows))
    #     print(A)
    #     r = columns

    #     m = np.random.choice(np.arange(r+1), size=r, replace=False)
    #     m = np.sort(m)
    #     print(m)

    #     for i in range(len(m)-1):
    #         # print(np.zeros(m[i+1]))
    #         # print(A[m[i], 0:m[i+1]])
    #         A[m[i], :m[i+1]] = np.zeros(m[i+1])

    #     A[m[-1], :m[-1]] = np.zeros(m[-1])

    #     # print(np.zeros(rows - m[-1]))

    elif type_ == 'degen':
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        A = np.zeros((rows,rows))
        n = np.random.randint(1, rows+1)
        # print(n)
        lamb = np.random.random(n)
        zer = np.sort(np.random.choice(np.arange(0, n), size=np.random.randint(1,n+1), replace=False))
        # print(lamb, zer)
        lamb[zer] = 0
        m = np.sort(np.random.choice(np.arange(1, rows), size=n-1, replace=False))
        # print(m)
        m = np.concatenate(([0], m))
        m = np.concatenate((m, [rows]))
        # print(m)
        for i in range(n):
            A[m[i]:m[i+1], m[i]:m[i+1]] = jordan(m[i+1]-m[i], lamb[i])
        
        B = np.random.random((rows,rows))
        return B@A@np.linalg.inv(B)
        # A = jordan(rows, 2)
    
    elif type_ == 'vozm':
        A = np.random.choice(np.linspace(-eps, eps, 1000), size=(rows, columns))

    return A

# matrix_generate(3, 2, type_='stup')
n = 10
A = matrix_generate(n,n, 'degen')
# print(A)
print(np.linalg.matrix_rank(A))

3


In [154]:
np.random.randint(1,3)

2

## Task 4.2

$$L_1, L_2, L_{\infty}$$

In [52]:
from scipy import linalg

def Norma(A, type_='inf'):
    norma = None
    if type_ == 'inf':
        if len(A.shape) == 1:
            norma = A.max()
        else:
            # B = np.sum(np.abs(A), axis=1).max()
            # print(B)
            norma = np.sum(np.abs(A), axis=1).max()

    if type_ == 'L1':
        if len(A.shape) == 1:
            norma = np.sum(np.abs(A))
        else:
            norma = np.sum(np.abs(A.T), axis=1).max()

    if type_ == 'L2':
        if len(A.shape) == 1:
            norma = np.sqrt(np.sum(A**2))
        else:
            norma = np.sqrt(linalg.eigvals(A.T@A).max())
    
    return norma

def chisl_obus(A, f, type_norm = 'inf'):
    return Norma(A, type_norm)*Norma(linalg.inv(A), type_norm)



# A = np.array([1, 2, 3])
A = np.array([[1, 2, 3], [4,5,6]])
Norma(A)

[ 6 15]


15

## Task 4.3

In [94]:
def gauss(A, slu=False):
    # print(b.reshape(-1, 1))
    # B = np.concatenate((A, b.reshape(-1, 1)), axis=1)
    # print(A)
    n = 0
    k = 0
    if slu: k = 1
    for i in range(A.shape[1]-k):
        flag = False
        for j in range(n, A.shape[0]):
            if A[j, i] != 0:
                flag = True
                if j == n:
                    break
                else:
                    A[[n, j], i] = A[[j, n], i]
                    break
        
        if flag:
            A[n, :] = A[n, :]/A[n, i]
            for j in range(A.shape[0]):
                if n != j:
                    A[j, :] -= A[n, :]*A[j, i]
        # print(A)
        n += 1

    return A

def rk(A):
    A = gauss(A)
    # print(A.sum(axis=1))
    A = A.sum(axis=1)
    n = A[A != 0].shape[0]
    return n
            

    # print(B)

# A = np.array([[3,2,-5],[2,-1,3],[1,2,-1]]).astype('float32')
# b = np.array([-1,13,9]).astype('float32')
# A = np.array([[4,-3,2,-1],[3,-2,1,-3],[5,-3,1,-8]]).astype('float32')
# b = np.array([8,7,1]).astype('float32')
# B = np.array([[7,-2,-1],[6,-4,-5],[1,2,4]]).astype('float16')
# b = np.array([2,3,5]).astype('float32')
# B = np.concatenate((A, b.reshape(-1, 1)), axis=1)
# B = np.array([[1,2,3,4],[0,1,-1,2],[0,1,-1,2],[0,2,-2,4]]).astype('float32')
# gauss(B, True)
rk(A)

2

In [105]:
def SLU(A, b):
    B = np.concatenate((A, b.reshape(-1, 1)), axis=1)
    B = gauss(B, True)[:, -1].reshape(-1)
    return B

type_ = 'float32'
A = np.array([[3,2,-5],[2,-1,3],[1,2,-1]]).astype(type_)
b = np.array([-1,13,9]).astype(type_)
SLU(A, b)

array([3.0000007, 5.       , 4.0000005], dtype=float32)

In [124]:
a = np.float64(0.2)
b = np.float64(0.1)
d = np.float(0.3)
print(d-(a+b))

-5.551115123125783e-17


C:\Users\vandy\AppData\Local\Temp\ipykernel_21800\3762870695.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  d = np.float(0.3)
